In [4]:
# use NUMBA!
import numpy as np
import cv2

# As Inputs
## Within each pitch, we vary:
- position of camera, given as xyz coordinates
- angle camera is viewing, given by thetax, thetay, and thetaz

## For each pitch, we vary
- launch speed
- angle of release




In [2]:
# represent all the Cs = np.array

In [72]:
src = np.zeros((6, 3))
src[:,1] = -2
src[:,2] = range(6) # source points
rvec = np.array([0,0,0], np.float) # rotation vector
tvec = np.array([0,0,-10], np.float) # translation vector

# from http://stackoverflow.com/questions/34963337/iphone-6-camera-calibration-for-opencv
# reversed x and y for landscape
fy=1229.0; cy=360.0; fx=1153.0; cx=640.0; 

cameraMatrix = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])
result = cv2.projectPoints(src, rvec, tvec, cameraMatrix, None)
for n in range(len(src)):
    print src[n], '==>', result[0][n]

[ 0. -2.  0.] ==> [[ 640.   605.8]]
[ 0. -2.  1.] ==> [[ 640.          633.11111111]]
[ 0. -2.  2.] ==> [[ 640.    667.25]]
[ 0. -2.  3.] ==> [[ 640.          711.14285714]]
[ 0. -2.  4.] ==> [[ 640.          769.66666667]]
[ 0. -2.  5.] ==> [[ 640.   851.6]]


In [80]:
src_h = np.hstack([src, np.ones(6).reshape(-1,1)])
p1, Rc =rotate(np.dot(CtoT(np.array([0,0,10])),src_h[0]))

t = -np.dot(Rc, np.array([0,0,10,1]))
t

array([ -0.,  -0., -10.,  -1.])

In [58]:
np.dot(
    np.dot(cameraMatrix,np.hstack([np.identity(3), np.zeros(3).reshape(-1,1)])), p1
    )

array([ 6400.,  6058.,    10.])

In [34]:
def rotate_x(theta_x):
    r1 = [1,0,0,0]
    r2 = [0, np.cos(theta_x), -np.sin(theta_x), 0]
    
    r3 = [0, np.sin(theta_x), np.cos(theta_x), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_y(theta_y):
    r1 = [np.cos(theta_y), 0, np.sin(theta_y), 0]
    r2 = [0, 1, 0, 0]
    
    r3 = [-np.sin(theta_y),0, np.cos(theta_y), 0]
    
    r4 = [0,0,0,1]
    return np.array([r1,r2,r3,r4])

def rotate_z(theta_z):
    r1 = [np.cos(theta_z), -np.sin(theta_z), 0, 0]
    r2 = [np.sin(theta_z), np.cos(theta_z), 0, 0 ]
    
    r3 = [0, 0, 1, 0]
    
    r4 = [0, 0, 0, 1]
    return np.array([r1,r2,r3,r4])

def rotate(p, thetas=(0,0,0), degrees=False, transpose=False):
    """ rotate x, then y, then z"""
    if degrees == True:
        thetas = [np.deg2rad(th) for th in thetas]
    X_rot_mat = rotate_x(thetas[0])
    Y_rot_mat = rotate_y(thetas[1])
    Z_rot_mat = rotate_z(thetas[2])
    if transpose:
        X_rot_mat = X_rot_mat.T
        Y_rot_mat = Y_rot_mat.T
        Z_rot_mat = Z_rot_mat.T
        
    rot_mat = np.dot(np.dot(Z_rot_mat, Y_rot_mat), X_rot_mat)
    return np.dot(rot_mat, p), rot_mat

    #return np.dot(np.dot(np.dot(rotate_x(thetas[0]), rotate_y(thetas[1])), rotate_z(thetas[2])), p)

def CtoT(C):
    """C is camera position, will return translation matrix"""
    
    return np.vstack([np.hstack([np.identity(3), C.reshape(-1,1)]), np.array([0,0,0,1])])


In [62]:
import math
np.rad2deg(math.atan(1/1200.)*2)

0.095492943750293213